### Libraries

In [16]:
%%capture
%reset -f                        # clear all variables from the workspace
'generic imports'
import os
import pandas as pd
import sys
sys.path.append(os.path.abspath('..'))
from src import utils
import importlib
importlib.reload(utils)
from psutil import virtual_memory    
import datetime
import numpy as np                 

'machine learning imports'
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn import metrics
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
print('TensorFlow version:', tf.__version__)

TensorFlow version: 2.12.0


### GPU

In [3]:
# if gpu available print name, else use cpu
if tf.test.is_gpu_available():
    print('GPU:', tf.test.gpu_device_name())
else:
    print('CPU:', tf.config.list_physical_devices('CPU'))

CPU: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


2023-10-29 20:19:23.158853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-29 20:19:23.159253: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-10-29 20:19:23.199048: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/t

### Load Data

In [4]:
# Define the augmentation method and the data directory
AUGMENTATION = 'SMOTE'
data_dir = os.path.abspath('../data')

# Load the train and test datasets
df_train, df_test = utils.load_dataset(data_directory=data_dir, 
                                       augmentation=AUGMENTATION, 
                                       ignore_columns=['mqtt.topic_0.0.1', 
                                                       'Unnamed: 0', 
                                                       'mqtt.topic_Temperature_and_Humidity', 
                                                       'mbtcp.unit_id', 
                                                       'mbtcp.trans_id'])   

Loading complete.
Training data: 1500000 rows, 98 columns. 
Test data: 381934 rows, 98 columns.


### Data Preparation

In [6]:
# Creates X_train, y_train
X_train = df_train.drop(['Attack_label', 'Attack_type'], axis=1)
y_train = df_train['Attack_type']

# Creates X_test, y_test
X_test = df_test.drop(['Attack_label', 'Attack_type'], axis=1)
y_test = df_test['Attack_type']

#### Convert categorical features to one-hot encoded features

In [40]:
# One-hot encode the training and test labels if needed
X_train_enc, X_test_enc = utils.one_hot_encode_categorical(X_train, X_test, random_state=42)

No categorical features found. Returning original datasets.


#### Label Encoding

In [7]:
y_train_enc, y_test_enc, le = utils.encode_labels(y_train, y_test)

y_train_bin = tf.keras.utils.to_categorical(y_train_enc)
y_test_bin = tf.keras.utils.to_categorical(y_test_enc)

Attack_type and encoded labels:

Backdoor                0
DDoS_HTTP               1
DDoS_ICMP               2
DDoS_TCP                3
DDoS_UDP                4
Fingerprinting          5
MITM                    6
Normal                  7
Password                8
Port_Scanning           9
Ransomware              10
SQL_injection           11
Uploading               12
Vulnerability_scanner   13
XSS                     14


In [13]:
# print type of each variable X_train, y_train, X_test, y_test
print(f"X_train type: {type(X_train)},\n y_train type: {type(y_train)},\n X_test type: {type(X_test)},\n y_test type: {type(y_test)}")

X_train type: <class 'pandas.core.frame.DataFrame'>,
 y_train type: <class 'pandas.core.series.Series'>,
 X_test type: <class 'pandas.core.frame.DataFrame'>,
 y_test type: <class 'pandas.core.series.Series'>


#### Standardization of Data

In [17]:
X_train_scaled, X_test_scaled = utils.scale_data(X_train, X_test, scaler_type='standard')

                   Mean       Std    
Train:            0.0000     0.9520  
Test:            22.0152    123.9567 



### Model Training

In [18]:
# Define the model
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(le.classes_), activation='softmax')) 

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 

# ReduceLROnPlateau callback
monitor = tf.keras.callbacks.ReduceLROnPlateau(monitor="loss",
                                               factor=0.3,
                                               mode="min",
                                               patience=10,
                                               verbose=1,
                                               min_lr=1e-8)

# Checkpoint callback                                                
checkpoint = ModelCheckpoint('../checkpoints/neural_net/best_model_multiclass.h5', 
                              monitor='loss', 
                              save_best_only=True)

In [19]:
# Shuffle training data
X_train, y_train_bin = shuffle(X_train_scaled, y_train_bin, random_state=42)

# Train the model
history = model.fit(X_train, 
                    y_train_bin, 
                    epochs=100, 
                    batch_size=512, 
                    callbacks=[monitor, checkpoint])

Epoch 1/100


### Model Evaluation

In [59]:
# predict probabilities for test set and get the index of the highest probability
predictions = tf.argmax(model.predict(X_test), axis=1)
predictions[:3]

11936/11936 [==============================] - 8s 698us/step


<tf.Tensor: shape=(3,), dtype=int64, numpy=array([ 1, 13, 13])>

In [60]:
# Calculate metrics 
accuracy = metrics.accuracy_score(tf.argmax(y_test, axis=1), predictions)
precision_m = metrics.precision_score(tf.argmax(y_test, axis=1), predictions, average='macro', zero_division=1)
recall_m = metrics.recall_score(tf.argmax(y_test, axis=1), predictions, average='macro')
f1_score_m = metrics.f1_score(tf.argmax(y_test, axis=1), predictions, average='macro')
precision_w = metrics.precision_score(tf.argmax(y_test, axis=1), predictions, average='weighted', zero_division=1)
recall_w = metrics.recall_score(tf.argmax(y_test, axis=1), predictions, average='weighted')
f1_score_w = metrics.f1_score(tf.argmax(y_test, axis=1), predictions, average='weighted')

print("Model Evaluation Metrics")
print("~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Accuracy: {:.2f}".format(accuracy))
print("Precision (macro): {:.2f}".format(precision_m))
print("Recall (macro): {:.2f}".format(recall_m))
print("F1 (macro): {:.2f}".format(f1_score_m))
print("Precision (weighted): {:.2f}".format(precision_w))
print("Recall (weighted): {:.2f}".format(recall_w))
print("F1 (weighted): {:.2f}".format(f1_score_w))
print("~~~~~~~~~~~~~~~~~~~~~~~~~")

InvalidArgumentError: Value for attr 'T' of string is not in the list of allowed values: float, double, int32, uint8, int16, int8, int64, bfloat16, uint16, half, uint32, uint64, qint8, quint8, qint32, qint16, quint16, bool
	; NodeDef: {{node ArgMax}}; Op<name=ArgMax; signature=input:T, dimension:Tidx -> output:output_type; attr=T:type,allowed=[DT_FLOAT, DT_DOUBLE, DT_INT32, DT_UINT8, DT_INT16, 5081812382442207599, DT_QUINT8, DT_QINT32, DT_QINT16, DT_QUINT16, DT_BOOL]; attr=Tidx:type,default=DT_INT32,allowed=[DT_INT16, DT_INT32, DT_INT64]; attr=output_type:type,default=DT_INT64,allowed=[DT_INT16, DT_UINT16, DT_INT32, DT_INT64]> [Op:ArgMax]

#### Save Metrics Results 

In [ ]:
# create dictionary for results
results = {
    "model": "Neural Net",
    "augmentations": AUGMENTATION,
    "timestamp": datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "accuracy": accuracy,
    "precision_macro": precision_m,
    "recall_macro": recall_m,
    "f1_macro": f1_score_m,
    "precision_weighted": precision_w,
    "recall_weighted": recall_w,
    "f1_weighted": f1_score_w
    }

# save results to csv   
utils.save_results_to_csv([results], '../results/metrics/neural_net.csv')

#### Confusion Matrix

In [ ]:
conf_mat = metrics.confusion_matrix(tf.argmax(y_test, axis=1), predictions)

attack_labels = ['Backdoor', 'DDoS_HTTP', 'DDoS_ICMP', 'DDoS_TCP', 'DDoS_UDP', 
'Fingerprinting', 'MITM', 'Normal', 'Password', 'Port_Scanning', 'Ransomware', 
'SQL_injection', 'Uploading', 'Vulnerability_scanner', 'XSS']

# Create a dataframe from the confusion matrix
conf_mat_df = pd.DataFrame(conf_mat, 
                            index = attack_labels, 
                            columns = attack_labels)
conf_mat_df.index.name = 'Actual'
conf_mat_df.columns.name = 'Predicted'


# Save the confusion matrix
conf_mat_df.to_csv(f"../results/conf_matrix/{results['model']}_{results['augmentations']}.csv")
conf_mat_df

Predicted,Backdoor,DDoS_HTTP,DDoS_ICMP,DDoS_TCP,DDoS_UDP,Fingerprinting,MITM,Normal,Password,Port_Scanning,Ransomware,SQL_injection,Uploading,Vulnerability_scanner,XSS
Actual,,,,,,,,,,,,,,,
Backdoor,4700,0,0,81,1,0,0,0,0,0,0,0,0,0,0
DDoS_HTTP,0,9171,0,0,0,0,0,0,0,0,0,0,0,5,452
DDoS_ICMP,0,0,13463,0,5,33,0,0,0,0,0,0,0,0,0
DDoS_TCP,0,0,0,10009,0,0,0,0,0,0,0,0,0,0,0
DDoS_UDP,0,0,0,0,24601,0,0,0,0,0,0,0,0,0,0
Fingerprinting,24,0,25,23,5,68,0,0,0,0,0,0,0,0,1
MITM,0,0,0,0,0,0,76,0,0,0,0,0,0,0,0
Normal,0,2,0,3,0,0,0,272766,0,0,0,0,0,4,1
Password,0,0,0,0,0,0,0,0,1768,0,0,7446,894,0,0
